In [1]:
import json
import importlib
import os
current_dir=os.getcwd()
os.chdir("../../")
# print(os.getcwd())
import dist_utilities as dist_ult
# import noise_est_utilities as ne_ult
importlib.reload(dist_ult)
os.chdir(current_dir)
import pickle
from qiskit.result import Result
from qiskit.visualization import plot_histogram, plot_distribution
import matplotlib.pyplot as plt
from matplotlib.axes import Axes
import numpy as np
from copy import deepcopy
from qiskit.result import QuasiDistribution

In [2]:
with open("ideal_counts_hardware.json", "r") as f:
    ideal_counts=json.load(f)
with open("noisy_counts_hardware.json", "r") as f:
    noisy_counts=json.load(f)
with open("noisy_noise_est_dist_hardware.json", "r") as f:
    noisy_noise_est_dist=json.load(f)
with open("ideal_noise_est_counts_hardware.json", "r") as f:
    ideal_noise_est_counts=json.load(f)
print("ideal: ", ideal_noise_est_counts)
print("noise est dist: ", noisy_noise_est_dist)
print("noisy counts: ", noisy_counts)
print("ideal counts: ", ideal_counts)
col_label=list(dict(ideal_noise_est_counts).keys())[0]
# corrected_counts=ne_ult.get_corrected_counts_from_counts(noisy_counts, noisy_noise_est_dist, list(ideal_noise_est_counts.keys())[0])

# print(corrected_counts)

ideal:  {'0010111100': 200000}
noise est dist:  {'0010111100': 0.590965, '0000111100': 0.025585, '0010011100': 0.029715, '1010111100': 0.08613, '1010111101': 0.00578, '0010111000': 0.0273, '0010100000': 0.000195, '1010110100': 0.00265, '0010100100': 0.00254, '0010101101': 0.003265, '0010110000': 0.002045, '0110001100': 0.00012, '0010011101': 0.002005, '0010110100': 0.018405, '1010101100': 0.006905, '0010111101': 0.04094, '0010111110': 0.01152, '0110111110': 0.000285, '0010101100': 0.043245, '0110111100': 0.0157, '1010011100': 0.0043, '1010111000': 0.00367, '0010000100': 0.00036, '0000101100': 0.00189, '0011110100': 0.00043, '0010101000': 0.001825, '0011111100': 0.014765, '1010110000': 0.000285, '1010111110': 0.001505, '0010111010': 0.00156, '0010010100': 0.00117, '1000111100': 0.003625, '0100111100': 0.001265, '0010101110': 0.000955, '0010101010': 0.00013, '0110111010': 4.5e-05, '1011111100': 0.002165, '0010001100': 0.00351, '0110101100': 0.00104, '0001101100': 9e-05, '0010011000': 0.0

In [3]:
# plot_histogram([ideal_counts, noisy_counts, corrected_counts], number_to_keep=5, legend=["ideal counts", "noisy counts", "corrected counts"], figsize=(20,20), filename="plot_sim.jpg")


In [4]:
num_qubits=len(list(noisy_counts.items())[0][0])

# if 2**10<2**num_qubits:
num_keep=2**15#num_qubits#2**10
if num_keep>2**num_qubits:
    num_keep=2**num_qubits
noisy_counts_orig=deepcopy(noisy_counts)
print("num keep ", num_keep)
noisy_noise_est_dist=dist_ult.convert_to_first_col(noisy_noise_est_dist, col_label)
noisy_noise_est_dist=dist_ult.filter_counts(noisy_noise_est_dist, num_keep)
noisy_counts=dist_ult.filter_counts(noisy_counts, num_keep)
all_keys, noisy_noise_est_dist, noisy_counts=dist_ult.fill_counts_range(noisy_noise_est_dist, noisy_counts, num_keep, num_qubits)

noisy_noise_est_vec=sorted(list(noisy_noise_est_dist.items()), key=lambda x: int(x[0], 2))
noisy_noise_est_vec=[elem[1] for elem in noisy_noise_est_vec]
counts_vec=sorted(list(noisy_counts.items()), key=lambda x: int(x[0], 2))
counts_vec=[elem[1] for elem in counts_vec]
# print(len(noisy_noise_est_vec))
# print(len(counts_vec))
quasi_corrected_counts=dist_ult.get_quasi_corrected(noisy_noise_est_vec, counts_vec)
quasi_corrected_counts=dist_ult.label_quasis(quasi_corrected_counts, all_keys)
quasi_method=QuasiDistribution(quasi_corrected_counts)
quasi_corrected_counts=quasi_method.nearest_probability_distribution().binary_probabilities(num_qubits)
# print(quasi_corrected_counts)
# quasi_corrected_counts={bin(idx)[2::].zfill(num_qubits):val for idx, val in enumerate(quasi_corrected_counts)}



num keep  1024
max exist  1020
min exist  0
size before  805
1024
1024


In [5]:
fig=plot_histogram([ideal_counts, noisy_counts_orig, quasi_corrected_counts], number_to_keep=10, 
               legend=["ideal", "raw", "corrected"], title="Dicke n=10, k=1 Quantum Hardware", bar_labels=False)
fig.savefig("plot_dicke10_1_counts_hardware_ibm_marrakesh.jpg", bbox_inches="tight")


In [6]:
fig=plot_distribution([ideal_counts, noisy_counts_orig, quasi_corrected_counts], number_to_keep=10, 
                  legend=["ideal", "raw", "corrected"], title="Dicke n=10, k=1 Quantum Hardware", bar_labels=False) #figsize=(15,15))
ax=fig.axes[0]
ax.set_ylabel("Probability")
fig.savefig("plot_dicke10_1_dist_hardware_ibm_marrakesh.jpg", bbox_inches="tight")

In [7]:
from qiskit.quantum_info import hellinger_fidelity
print(hellinger_fidelity(ideal_counts, quasi_corrected_counts))
print(hellinger_fidelity(ideal_counts, noisy_counts_orig))
print(ideal_counts)
print(quasi_corrected_counts)
print(noisy_counts_orig)

0.9349764027511057
0.57638454664422
{'1000000000': 20000, '0100000000': 20000, '0010000000': 20000, '0001000000': 20000, '0000100000': 20000, '0000010000': 20000, '0000001000': 20000, '0000000100': 20000, '0000000010': 20000, '0000000001': 20000}
{'1100001000': 1563.21792121378, '1100000001': 1573.10913894624, '1100100000': 1580.33787402579, '1100000010': 1594.12549508329, '1100010000': 1605.36582091784, '1100000100': 1640.97571211909, '1101000000': 1662.76565939820, '1110000000': 1714.89733605472, '0000000001': 17220.3110985539, '0000100000': 17887.6268819556, '0010000000': 18385.1776377155, '0000001000': 18587.9609354097, '0000000010': 18804.4871737488, '0000000100': 18873.5602247067, '0001000000': 18932.3921787066, '1000000000': 18948.4281715146, '0000010000': 19640.9175811428, '0100000000': 19784.3431587870}
{'1110000010': 124, '0001010000': 1172, '0000000010': 11519, '1100000010': 1359, '0000000001': 10774, '1000000001': 1442, '0000000000': 5962, '0000011000': 1181, '1000001010': 